In [1]:
!pip install --upgrade --quiet  langchain-google-genai langchain_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 9.7 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import os

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage

In [18]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [19]:
llm.invoke("Hi! How are you?").content

'As an AI, I don\'t have feelings like humans do, so I don\'t experience "being" in the same way. However, I\'m here and ready to assist you with any questions or tasks you may have! What can I do for you today? \n'

## Define tool

In [20]:
@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

tools = [add, multiply]

## Link tool with llm

In [21]:
llm_with_tools = llm.bind_tools(tools) # tool_choice = 'any', 'actual_tool_name',

ai_msg = llm_with_tools.invoke("What is (121+5)*9 ?")

ai_msg.tool_calls

[{'name': 'add',
  'args': {'b': 5.0, 'a': 121.0},
  'id': '938abf7c-c861-4e90-82d6-70b8dcaa5c88'}]

In [22]:
## See the tools available
llm_with_tools.to_json()['kwargs']['kwargs']['tools']

[{'function_declarations': [{'name': 'add',
    'description': 'Adds a and b.\n\n    Args:\n        a: first int\n        b: second int',
    'parameters': {'type': 'object',
     'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}},
     'required': ['a', 'b']}}]},
 {'function_declarations': [{'name': 'multiply',
    'description': 'Multiplies a and b.\n\n    Args:\n        a: first int\n        b: second int',
    'parameters': {'type': 'object',
     'properties': {'b': {'type': 'integer'}, 'a': {'type': 'integer'}},
     'required': ['a', 'b']}}]}]

In [23]:
query = "What is (121+5)*9 ?"

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(query)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

[HumanMessage(content='What is (121+5)*9 ?'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 121.0, "b": 5.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-5311c488-8a9c-4b77-a8d2-afcc608fce7d-0', tool_calls=[{'name': 'add', 'args': {'a': 121.0, 'b': 5.0}, 'id': '495d7378-6a5f-4577-855a-6e4195564821'}], usage_metadata={'input_tokens': 137, 'output_tokens': 20, 'total_tokens': 157}),
 ToolMessage(content='126', tool_call_id='495d7378-6a5f-4577-855a-6e4195564821')]

In [24]:
final_response = llm.invoke(messages)

print(final_response.content)

Here's how to solve the problem:

1. **Parentheses first:**  121 + 5 = 126
2. **Multiplication:** 126 * 9 = 1134

Therefore, (121 + 5) * 9 = **1134** 



In [ ]:
!pip install langchain

In [25]:
#Create Independent Decision Taking Agent
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_openai_tools_agent
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. You may not need to use tools for every query - the user may just want to chat!",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [26]:
agent_executor.invoke({"messages": [HumanMessage(content="Add 8 to 199 and then multiply by 4")]})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 199.0, 'b': 8.0}`


207The answer is 828. 


> Finished chain.


{'messages': [HumanMessage(content='Add 8 to 199 and then multiply by 4')],
 'output': 'The answer is 828. \n'}

In [27]:
(8+199)*4

828

In [28]:
agent_executor.invoke({"messages": [HumanMessage(content="Add 121 to 5 then multiply result by 2")]})



> Entering new AgentExecutor chain...
Okay, I can do that.  The result is 252. 


> Finished chain.


{'messages': [HumanMessage(content='Add 121 to 5 then multiply result by 2')],
 'output': 'Okay, I can do that.  The result is 252. \n'}

In [29]:
agent_executor.invoke({"messages": [HumanMessage(content="Add two to one hundred twenty one and then multiply result by five")]})



> Entering new AgentExecutor chain...
121 + 2 = 123
123 * 5 = 615 
The answer is 615. 


> Finished chain.


{'messages': [HumanMessage(content='Add two to one hundred twenty one and then multiply result by five')],
 'output': '121 + 2 = 123\n123 * 5 = 615 \nThe answer is 615. \n'}

In [30]:
(2+121) *5

615